In [ ]:
import requests
from lxml import html
from lxml import etree
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd

from io import StringIO, BytesIO
import datetime
from datetime import date
from datetime import timedelta, datetime
import os

import json

In [ ]:

#from requests_base import RequestsPage


#####Pages######
##base
class RequestsPage:
        
    def get_html(self, url, payload=None):
        self.url = url
        fakeuser = UserAgent()
        headers = {'user-agent': fakeuser.chrome}
        data = requests.get(url, headers=headers, params=payload)
        #print(data.url)
        return data

    def save_html(self, data, filename):
        self.file = open(filename, "a")
        self.file.write(data)
        self.file.close()
        
##config

class ShelbyURLs:
    #case_details = 'https://gscivildata.shelbycountytn.gov/pls/gnweb/ck_public_qry_doct.cp_dktrpt_docket_report?backto=C&case_id=2061459&begin_date=&end_date='
    #case_details = 'https://gscivildata.shelbycountytn.gov/pls/gnweb/ck_public_qry_doct.cp_dktrpt_docket_report?backto=C&case_id={}&begin_date=&end_date='
    case_details = 'https://gscivildata.shelbycountytn.gov/pls/gnweb/ck_public_qry_doct.cp_dktrpt_docket_report?'





In [ ]:
class NoDateDiscoveryMixIn():
    
    
    def _get_today_date(self):
        self.current_date = datetime.today().date()
        
    
    def _clean_date(self, date):
        self.date = date[0].split(", ")
        self.final_date = f'{self.date[2].strip()}-{self.date[1].split()[0]}-{self.date[1].split()[1].strip()[0:2]}'
        self.clean_date = datetime.strptime(self.final_date, '%Y-%B-%d').date()
        return(self.clean_date)

  
    def _create_tree(self):
        self.parser = etree.HTMLParser()
        self.tree = etree.parse(StringIO(self.output.text), self.parser)
    
    def _locators(self):
        #self.case_present = self.tree.xpath('//*[contains(text(), "No case was found")]/text()')
        #self.date = self._clean_date(self.tree.xpath('//table[2]//tr[2]//td[3]//text()'))
        pass
        
        
        
  
    def _search_no_case(self):
        
        self.case_present = self.tree.xpath(self.case_status)
        ls_dates = []
        counter = 0   
        while len(self.case_present) == False:
            counter = counter +1
            date = self._clean_date(self.tree.xpath(self.filing_date))
            
            daily_average_filings = 40

            if counter != 1:
                latest_date = ls_dates.pop()
                if ((date-latest_date).days)<0:
                    date = latest_date
                    
            ls_dates.append(date)
            days_between = (self.current_date - date).days

            if days_between < 10:
                daily_average_filings = 10
            if days_between == 0:
                days_between = 1

            case_increment = days_between * daily_average_filings
            self.case_number = self.case_number + case_increment
            self.page_source("Shelby", self.case_number)
            self._create_tree()
            self.case_present = self.tree.xpath(self.case_status)

           
        #pass

    def _search_for_case(self):
        
        while len(self.case_present) == True:
            self.case_number = self.case_number-1
            self.page_source("Shelby", self.case_number)
            self._create_tree()
            self.case_present = self.tree.xpath(self.case_status)
            
    def manuall_site_param(self):
        raise DumbDeveloperException('This function requires a manual script. Are you a developer? Did you forget to overwrite manual_site_parm()')
    
        
            
                
   
    def most_recent_case(self, start_case, filing_date, case_status, driver=None):
        
        self.filing_date = filing_date
        self.case_status = case_status
        self.start_case = start_case
        self._get_today_date()
        self.page_source("Shelby", self.start_case)
        self._create_tree()
        self._search_no_case()
        self._search_for_case()
        print(f'most recent case is: {self.case_number}')
        return(self.case_number)
       
        #self.clean_date = self._clean_date()

In [ ]:
##search_page
class SearchPage(RequestsPage, NoDateDiscoveryMixIn):
    
    
    
    def _build_payload(self):
        self.payload = {
            'backto' : "C",
            'case_id':  self.case_number,
            'begin_date': '',
            'end_date' : ''
                       }
    
    
    def page_source(self, county, case_number):
        self.county = county
        
        self.case_number = case_number
        self._build_payload()
        self.url = ShelbyURLs.case_details.format(self.case_number)
        
        self.output = self.get_html(self.url, payload = self.payload)
        #print(self.url)
        #print("+++")
        #print(self.output.url)
        
        #self.save_html(self.output, "test2")
        
    def most_recent_case(self, start_case):
        super().most_recent_case(start_case ,'//table[2]//tr[2]//td[3]//text()','//*[contains(text(), "No case was found")]/text()' )

In [ ]:
class Parser():
    def __init__(self):
        self.lxml_parser = etree.HTMLParser()

    
    def open_html(self, html_file):
        self.html_file = html_file
        self.html = open(self.html_file, "r")
        self.html = self.html.read()
        #lxml will not access text after <br> tags and xpaths to their location return nothing, thus removal solves the problem with no known drawbacks
        self.html = self.html.replace('<BR>', ' ')
        #print(self.html)
        return etree.parse(StringIO(self.html), self.lxml_parser)
    
    def _get_info(self, location):
        self.location = location
        #print("+++++++")
        #print(self.tree.xpath(f'{self.location}//text()'))
        #print("++++++++")
        return self.tree.xpath(self.location)
    """
    def _get_info2(self, location):
        self.location = location
        #print("+++++++")
        #print(self.tree.xpath(f'{self.location}//text()'))
        #print("++++++++")
        return self.tree.xpath(f'{self.location}//text()')
    """
    
    def clean(self, dirty):
        self.dirty = dirty
        self.dirty = self.dirty.lower().strip().replace('\n', '')
        return self.dirty

    def find_element_by_xpath(self, location):
        self.info = self._get_info(location)
        if isinstance(self.info, list):
            if len(self.info) == 0:
                #print('length is 0')
                return 'none'
            elif self.info[0].text != None and self.clean(self.info[0].text) != '':
                #print('position 1 is not none')
                return self.clean(self.info[0].text)
            else:
                #print('should be returning string of none')
                return 'none'
        elif self.info == None:
            #print('not a list, is none')
            return 'none'
        else:
            #print('not a list, not none')
            return self.info
    
    def find_elements_by_xpath(self, location):
        self.info = self._get_info(location)
        return self.info
        if isinstance(self.info, list):
            if len(self.info) == 1:
                raise ElementException('Element only returned one element. Try find_element_by_xpath (singular)')
            else:
                return self.info
        else:
            raise ElementException('Element only returned one element. Try find_element_by_xpath (singular).')
    
    #counts elements returned by find_elements_by_xpath, returns number
    def get_length_by_xpath(self, location):
        self.info = self._get_info(location)
        return len(self.info)
    
    # this function writes multiples 'none's to a given list until that list is 3 items long
    def expand_list(self, list, length):
        self.list = list
        self.length = length
        while len(self.list) < self.length:
            try:
                self.list.append('none')
            except:
                pass
        return self.list

    def more_than(self, list, count):
        self.list = list
        self.count = count
        if len(self.list) > self.count:
            return 'yes'
        else:
            return 'no'
        
    def text_between(self, text, before, after):
        self.text = text
        self.before = before
        self.after = after
        try:
            self.parse = self.text.split(self.before)
            self.parse = self.parse[1].split(self.after)
            self.parse = self.clean(self.parse[0])
        except:
            self.parse = 'none'
        return self.parse

    def does_string_appear(self, text, string):
        self.text = text
        self.string = string
        self.test = self.text.find(self.string)
        if self.test != -1:
            return 'yes'
        else:
            return 'no'
    
    #this function provides an entry_point to the best xpath logic for new users
    #it follows this logic{div/table item is in}/{div/html element text is in}/{text to search}/{xpath to follow to get to relative field}
    def string_search(self, text, table, route=None, text_location=None):
        self.text = text
        self.table = table
        self.route = route
        self.text_location = text_location
        if self.text_location == None and self.route == None:
            #('no location and no route')
            return self.find_element_by_xpath(f'{self.table}//*[contains(text(), "{text}")]')
        elif self.text_location == None and self.route != None:
            #('no location')
            return self.find_element_by_xpath(f'{self.table}//*[contains(text(), "{text}")]/{self.route}')
        elif self.text_location != None and self.route == None:
            #('no route')
            return self.find_element_by_xpath(f'{self.table}//{self.text_location}[contains(text(), "{text}")]')
        elif self.text_location != None and self.route != None:
            #('location and route')
            return self.find_element_by_xpath(f'{self.table}//{self.text_location}[contains(text(), "{text}")]/{self.route}')
    
    
    # this function creates a new database and failure log but wont overwrite existing one
    def create_csv(self, database_name, header):
        self.database_name = database_name
        self.header = header
        self.file = f'{file_path}/{self.database_name}'
        if path.exists(self.file):
            print('CSV already exists.')
        else:
            with open(self.file, 'w', newline='') as self.outfile:
                self.writer = csv.writer(self.outfile)
                self.writer.writerow(self.header)
            print('New CSV created.')
    
    def write_data(self, data_out, database_name):
        #writing data to csv
        self.data_out = data_out
        self.database_name = database_name
        self.file = f'{file_path}/{self.database_name}'
        with open(self.file, 'a', newline='') as self.outfile:
            self.writer = csv.writer(self.outfile, delimiter=',')
            self.writer.writerow(self.data_out)
            
    def write_json_data(self, data_out, database_name):
        #writing data to json
        self.data_out = data_out
        self.database_name = database_name
        self.file = f'{file_path}/{self.database_name}'
        with open(self.file, 'w') as self.outfile:
            json.dump(self.data_out, self.outfile)
    
    

In [ ]:
class ShelbyParser(Parser):
    
    
        
    def shelby_eviction(self, tree):
        self.tree = tree
        
        #Initialize storage containers
        self.case_object = dict()
        self.party_obj = []
        self.docket_obj = []
        
        ##table locations
        self.case_description_table = '//*[contains(text(), "Case Description")]/parent::node()/parent::node()//following-sibling::table[1]'
        self.report_selection_criteria_table = '//*[contains(text(), "Report Selection Criteria")]/parent::node()/parent::node()//following-sibling::table[1]' 
        self.case_parties_table = '//*[contains(text(), "Case Parties")]/parent::node()//following-sibling::table[1]'
        self.docket_entries_table = '//*[contains(text(), "Docket Entries")]/parent::node()/parent::node()//following-sibling::table[1]'
        self.get_horizontal_info = '/parent::node()//following-sibling::td[1]'
        self.rows_route = 'descendant::tr'
        
        ##Horizontal information
        #print(f'Horizontal Info -----------------------------------------------------------------------')
        self.case_id = self.string_search("Case ID", self.report_selection_criteria_table, route = self.get_horizontal_info, text_location = 'b')
        self.filing_date = self.string_search("Filing Date", self.case_description_table, route = self.get_horizontal_info, text_location = 'b')
        self.case_type = self.string_search("Type", self.case_description_table, route = self.get_horizontal_info, text_location = 'b')
        self.status = self.string_search("Status", self.case_description_table, route = self.get_horizontal_info, text_location = 'b')
        
        #print(f'Case: {self.case_id} and {self.type}')
        ##Vertical Information
        
        
        ####Case Parties
        #print(f'Case Partieeeeeees -----------------------------------------------------------------------')
        self.case_parties_row_locator = f'{self.case_parties_table}/{self.rows_route}'        
        self.rows = self.get_length_by_xpath(f'{self.case_parties_row_locator}')
                
        self.info_rows_index = list(range(1,self.rows))
        self.info_row_index_grouped = list(zip(*[iter(self.info_rows_index)]*3))
        
        
        for index in self.info_row_index_grouped:
            self.index = index  
            #plus 1 to account for first row being headers
            self.first_row = index[0]+1
            self.second_row = index[1]+1
            
    
            self.xpath1 = f'{self.case_parties_row_locator}[{self.first_row}]/td'
            self.xpath2 =  f'{self.case_parties_row_locator}[{self.second_row}]/td'


            self.seq = self.find_element_by_xpath(f'{self.xpath1}[1]')
            self.assoc = self.find_element_by_xpath(f'{self.xpath1}[2]')
            self.expn_date = self.find_element_by_xpath(f'{self.xpath1}[3]')
            self.type = self.find_element_by_xpath(f'{self.xpath1}[4]')
            self.id = self.find_element_by_xpath(f'{self.xpath1}[5]')
            self.parties_name = self.find_element_by_xpath(f'{self.xpath1}[6]')
            
            self.address = self.find_element_by_xpath(f'{self.xpath2}[2]')
            self.aliases = self.find_element_by_xpath(f'{self.xpath2}[4]')
            
            self.party_dict = {"seq":self.seq , "assoc":self.assoc,"expnDate":self.expn_date,"type":self.type,"id":self.id ,"name":self.parties_name, "address":self.address,"aliases" :self.aliases } 
            self.party_obj.append(self.party_dict)

        

        ####Docket Entries
       # print(f'Docket Entrieeeessss -----------------------------------------------------------------------')
        self.docket_entries_row_locator = f'{self.docket_entries_table}/{self.rows_route}'        
        self.rows = self.get_length_by_xpath(f'{self.docket_entries_row_locator}')
                
        self.info_rows_index = list(range(1,self.rows))
        self.info_row_index_grouped = list(zip(*[iter(self.info_rows_index)]*3))
        
        
        for index in self.info_row_index_grouped:
            self.index = index  
            #plus 1 to account for first row being headers
            self.first_row = index[0]+1
            self.second_row = index[1]+1
            
    
            self.xpath1 = f'{self.docket_entries_row_locator}[{self.first_row}]/td'
            self.xpath2 =  f'{self.docket_entries_row_locator}[{self.second_row}]/td'


            self.filingDate = self.find_element_by_xpath(f'{self.xpath1}[1]')
            self.description = self.find_element_by_xpath(f'{self.xpath1}[2]')
            self.docket_name = self.find_element_by_xpath(f'{self.xpath1}[3]')
            self.monetary = self.find_element_by_xpath(f'{self.xpath1}[4]')
            
            
            self.entry = self.find_element_by_xpath(f'{self.xpath2}[2]')
            
            
            self.docket_dict = {"filingDate": self.filingDate,  "description":self.description, "name": self.docket_name, "monetary": self.monetary, "entry":self.entry}

            self.docket_obj.append(self.docket_dict)
            
        #Compile final data
        self.case_object["case_id"] = self.case_id        
        self.case_object["filing_date"] = self.filing_date
        self.case_object["type"] = self.case_type
        self.case_object["status"] = self.status
        self.case_object["case_parties"] = self.party_obj
        self.case_object["docket_entries"] = self.docket_obj
        
        #print(self.case_object)
        return(self.case_object)
        


        

    

In [ ]:
# This searches for the most recent case number when given a previously exisiting case number
search = SearchPage()
search.most_recent_case(2061459)

In [ ]:
start_case = 2074248
ls_urls = []
for case_number in range(start_case,last_case+1):
    url = f'https://gscivildata.shelbycountytn.gov/pls/gnweb/ck_public_qry_doct.cp_dktrpt_docket_report?backto=C&case_id={case_number}&begin_date=&end_date='
    ls_urls.append(url)
    


In [ ]:
### Load documents into array after they are downloaded

parse = ShelbyParser()

ls_html = []
path="C:/Users/nicho/Documents/Code/howard-center-webscrapers2/Shelby/" 

files = os.listdir(path)
for file in files:
    ls_html.append(parse.open_html(f'{path}/{file}'))
print("done")



In [ ]:
# Write out parsed court cases into json
caseArr = []

count = 0
for case in ls_html:
    count = count + 1

    result = parse.shelby_eviction(case)
    caseArr.append(result)
    #clear_output(wait=True)
    print(count)

parse.write_json_data(caseArr, "shelby2020.json")
